# Movie context data clean up

In [120]:
import pandas as pd
import json

In [121]:
# depth of this file in the project
file_depth = '../..'

In [122]:
with open(file_depth + '/config/data_25m_config.json') as config_file:
    config = json.load(config_file)

In [5]:
new_context_path = config['new_context_path']
movies_with_context = pd.read_csv(file_depth + new_context_path + 'merged_movies_with_context.csv', encoding="UTF-8")
movies_with_context

,movieId,title,genresMovieLens,movieYear,imdbId,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genresImdb,directors,writers,actor
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,tt0114709,movie,Toy Story,0,1995,81,"Adventure,Animation,Comedy",nm0005124,"nm0005124,nm0230032,nm0004056,nm0710020,nm0923...","nm0000741,nm0001652,nm0001815,nm0029460,nm0039..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995.0,tt0113497,movie,Jumanji,0,1995,104,"Adventure,Comedy,Family",nm0002653,"nm0378144,nm0852430,nm0833164,nm0885575","nm0001372,nm0001564,nm0002123,nm0003742,nm0051..."
2,3,Grumpier Old Men (1995),Comedy|Romance,1995.0,tt0113228,movie,Grumpier Old Men,0,1995,101,"Comedy,Romance",nm0222043,nm0425756,"nm0025908,nm0132587,nm0153170,nm0160648,nm0263..."
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995.0,tt0114885,movie,Waiting to Exhale,0,1995,124,"Comedy,Drama,Romance",nm0001845,"nm0573334,nm0060103","nm0001365,nm0005375,nm0119698,nm0171213,nm0176..."
4,5,Father of the Bride Part II (1995),Comedy,1995.0,tt0113041,movie,Father of the Bride Part II,0,1995,106,"Comedy,Family,Romance",nm0796124,"nm0352443,nm0329304,nm0583600,nm0796124","nm0003028,nm0015233,nm0030926,nm0042963,nm0056..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62307,209157,We (2018),Drama,2018.0,tt6671244,movie,Wij,0,2018,100,Drama,nm1415482,"nm1415482,nm1818234","nm10312234,nm11039405,nm1963018,nm3760035,nm42..."
62308,209159,Window of the Soul (2001),Documentary,2001.0,tt0297986,movie,Window of the Soul,0,2001,73,Documentary,"nm0142504,nm1065588","nm0142504,nm1065588",nm1546270
62309,209163,Bad Poems (2018),Comedy|Drama,2018.0,tt6755366,movie,Bad Poems,0,2018,97,"Comedy,Drama",nm2520391,nm2520391,"nm0468514,nm0598406,nm10676258,nm1387171,nm158..."
62310,209169,A Girl Thing (2001),(no genres listed),2001.0,tt0249603,tvMovie,A Girl Thing,0,2001,237,"Comedy,Drama,Romance",nm0003022,nm0003022,"nm0121700,nm0453858,nm0841512"


### Fix movieYear column from title column

In [6]:
# movies_with_context['movieYear'] = movies_with_context['title'].str.extract(r'\((\d{4})\)')

print('Does any movie have no year informatrion? : ' + str(movies_with_context['movieYear'].isnull().values.any()))
print('Unique years: ')
print(movies_with_context['movieYear'].unique())

Does any movie have no year informatrion? : True
Unique years: 
[1995. 1994. 1996. 1976. 1992. 1988. 1967. 1993. 1964. 1977. 1965. 1982.
 1990. 1991. 1989. 1937. 1940. 1969. 1981. 1973. 1970. 1960. 1955. 1959.
 1968. 1980. 1975. 1986. 1948. 1943. 1950. 1946. 1987. 1997. 1974. 1956.
 1958. 1949. 1972. 1998. 1933. 1952. 1951. 1957. 1961. 1954. 1934. 1944.
 1963. 1942. 1941. 1953. 1939. 1947. 1945. 1938. 1935. 1936. 1926. 1932.
 1985. 1979. 1971. 1978. 1966. 1962. 1983. 1984. 1931. 1922. 1999. 1927.
 1929. 1930. 1928. 1925. 1914. 2000. 1919. 1923. 1920. 1918. 1921. 2001.
 1924. 2002. 2003. 1915. 2004. 1916. 1917. 2005. 2006. 1902. 1903. 2007.
 2008. 2009. 1912. 2010.   nan 1913. 2011. 1898. 1899. 1894. 2012. 1910.
 2013. 1896. 2014. 2015. 1895. 1909. 1911. 1900. 2016. 2017. 2018. 2019.
 1905. 1904. 1891. 1892. 1908. 1897. 1887. 1888. 1890. 1878. 1874. 1901.
 1907. 1906. 1883. 1880.]


In [7]:
print('Number of rows where there is no movieYear: ', movies_with_context['movieYear'].isnull().sum())
print('Number of rows where there is \"\\N\" as movieYear: ', (movies_with_context['movieYear'] == '\\N').sum())
print('Number of rows where there is no startYear: ', movies_with_context['startYear'].isnull().sum())
print('Number of rows where there is \"\\N\" as startYear: ', (movies_with_context['startYear'] == '\\N').sum())

Number of rows where there is no movieYear:  410
Number of rows where there is "\N" as movieYear:  0
Number of rows where there is no startYear:  0
Number of rows where there is "\N" as startYear:  8


There are 410 movies with no year value in title column. However, we can extract the year from column startYear for those rows where the movieYear is missing

In [8]:
movies_with_context['movieYear'].fillna(movies_with_context['startYear'], inplace=True)

In [9]:
print('Number of rows where there is no movieYear: ', movies_with_context['movieYear'].isnull().sum())
print('Number of rows where there is \"\\N\" as movieYear: ', (movies_with_context['movieYear'] == '\\N').sum())

Number of rows where there is no movieYear:  0
Number of rows where there is "\N" as movieYear:  2


We went from 410 missing values to only 2 missing movieYear values.

---
### Check MovieLens genre column 

In [10]:
print('Number of rows in genresMovieLens column where there is no genre: ', movies_with_context['genresMovieLens'].isnull().sum())
print('Number of rows in genresMovieLens column where there is \"(no genres listed)\" as genre: ', (movies_with_context['genresMovieLens'] == '(no genres listed)').sum())
print('Number of rows in genresMovieLens column where there is \"\\N\" as genre: ', (movies_with_context['genresMovieLens'] == '\\N').sum())

Number of rows in genresMovieLens column where there is no genre:  0
Number of rows in genresMovieLens column where there is "(no genres listed)" as genre:  5050
Number of rows in genresMovieLens column where there is "\N" as genre:  0


There are 5050 rows with text "(no genres listed)" as genre. We can replace this text with genre values from genresImdb column.

In [11]:
print('Number of rows in genresImdb column where there is no genre: ', movies_with_context['genresImdb'].isnull().sum())
print('Number of rows in genresImdb column where there is \"(no genres listed)\" as genre: ', (movies_with_context['genresImdb'] == '(no genres listed)').sum())
print('Number of rows in genresImdb column where there is \"\\N\" as genre: ', (movies_with_context['genresImdb'] == '\\N').sum())

Number of rows in genresImdb column where there is no genre:  0
Number of rows in genresImdb column where there is "(no genres listed)" as genre:  0
Number of rows in genresImdb column where there is "\N" as genre:  58


As we can see above, there are 58 row with value "\N" representing missing values in genresImdb column. Since it is a lower amount, we can replace the "(no genres listed)" with NaN and then fill the NaN values with the genre values from genresImdb column.

In [12]:
movies_with_context['genresMovieLens'] = movies_with_context['genresMovieLens'].replace('(no genres listed)', pd.NA)
movies_with_context['genresMovieLens'].fillna(movies_with_context['genresImdb'], inplace=True)

In [13]:
print('Number of rows in genresMovieLens column where there is no genre: ', movies_with_context['genresMovieLens'].isnull().sum())
print('Number of rows in genresMovieLens column where there is \"(no genres listed)\" as genre: ', (movies_with_context['genresMovieLens'] == '(no genres listed)').sum())
print('Number of rows in genresMovieLens column where there is \"\\N\" as genre: ', (movies_with_context['genresMovieLens'] == '\\N').sum())

Number of rows in genresMovieLens column where there is no genre:  0
Number of rows in genresMovieLens column where there is "(no genres listed)" as genre:  0
Number of rows in genresMovieLens column where there is "\N" as genre:  16


We went from 5050 missing values to 16 missing genre values. 

In [14]:
# year was saved as float, so we need to convert it to string and remove the .0 from the end
movies_with_context['movieYear'] = movies_with_context['movieYear'].astype(str).replace('\.0$', '', regex=True)
movies_with_context

,movieId,title,genresMovieLens,movieYear,imdbId,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genresImdb,directors,writers,actor
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,tt0114709,movie,Toy Story,0,1995,81,"Adventure,Animation,Comedy",nm0005124,"nm0005124,nm0230032,nm0004056,nm0710020,nm0923...","nm0000741,nm0001652,nm0001815,nm0029460,nm0039..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,tt0113497,movie,Jumanji,0,1995,104,"Adventure,Comedy,Family",nm0002653,"nm0378144,nm0852430,nm0833164,nm0885575","nm0001372,nm0001564,nm0002123,nm0003742,nm0051..."
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,tt0113228,movie,Grumpier Old Men,0,1995,101,"Comedy,Romance",nm0222043,nm0425756,"nm0025908,nm0132587,nm0153170,nm0160648,nm0263..."
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,tt0114885,movie,Waiting to Exhale,0,1995,124,"Comedy,Drama,Romance",nm0001845,"nm0573334,nm0060103","nm0001365,nm0005375,nm0119698,nm0171213,nm0176..."
4,5,Father of the Bride Part II (1995),Comedy,1995,tt0113041,movie,Father of the Bride Part II,0,1995,106,"Comedy,Family,Romance",nm0796124,"nm0352443,nm0329304,nm0583600,nm0796124","nm0003028,nm0015233,nm0030926,nm0042963,nm0056..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62307,209157,We (2018),Drama,2018,tt6671244,movie,Wij,0,2018,100,Drama,nm1415482,"nm1415482,nm1818234","nm10312234,nm11039405,nm1963018,nm3760035,nm42..."
62308,209159,Window of the Soul (2001),Documentary,2001,tt0297986,movie,Window of the Soul,0,2001,73,Documentary,"nm0142504,nm1065588","nm0142504,nm1065588",nm1546270
62309,209163,Bad Poems (2018),Comedy|Drama,2018,tt6755366,movie,Bad Poems,0,2018,97,"Comedy,Drama",nm2520391,nm2520391,"nm0468514,nm0598406,nm10676258,nm1387171,nm158..."
62310,209169,A Girl Thing (2001),"Comedy,Drama,Romance",2001,tt0249603,tvMovie,A Girl Thing,0,2001,237,"Comedy,Drama,Romance",nm0003022,nm0003022,"nm0121700,nm0453858,nm0841512"


---

Now that there is no need for imdbId, primaryTitle, genre and startYear column from Imdb dataset, we can drop it. We will also rename column genresMovieLens beck to genres.

In [108]:
movies_with_context.drop(columns=['imdbId', 'primaryTitle', 'genresImdb', 'startYear'], inplace=True)
movies_with_context.rename(columns={'genresMovieLens': 'genres'}, inplace=True)

-----------
-----------
### Delete rows

2 missing values in movieYear column:

In [109]:
zero_values_year = movies_with_context[movies_with_context['movieYear'] == '\\N']
zero_values_year

,movieId,title,genres,movieYear,titleType,isAdult,runtimeMinutes,directors,writers,actor
48645,177265,Checkmate,\N,\N,movie,0,\N,\N,\N,NaN
54941,190961,Poor Boy,Drama,\N,movie,0,\N,nm0001602,nm0131675,nm1862651


16 missing values in genre column:

In [110]:
zero_values_genres = movies_with_context[movies_with_context['genres'] == '\\N']
zero_values_genres

,movieId,title,genres,movieYear,titleType,isAdult,runtimeMinutes,directors,writers,actor
28427,131775,Mr. Civil Rights: Thurgood Marshall and the NA...,\N,2014,movie,0,57,nm0134890,\N,"nm1594239,nm3989370"
33154,142610,Un mundo cuadrado (2012),\N,2012,movie,0,\N,nm1723172,"nm1723172,nm2200603","nm1670234,nm3017005,nm3084953,nm3905620,nm3906..."
37678,153274,Langsamer Sommer (1976),\N,1976,movie,0,84,nm0683696,"nm1104332,nm0683696","nm1072960,nm1073484,nm1073604,nm3195878"
38315,154652,Life Lesson (1995),\N,1995,movie,0,105,nm0499618,nm0499618,"nm0087927,nm0130689,nm0515511,nm0535442,nm0771..."
38409,154870,Buried in Light (1994),\N,1994,video,0,60,nm0169486,\N,NaN
42481,164089,"Karagoez catalogo 9,5 (1983)",\N,1983,movie,0,54,"nm0316214,nm0723187",\N,NaN
43030,165273,That Was Rock (1984),\N,1984,video,0,\N,"nm0004596,nm0670282,nm0200597",\N,NaN
44369,168228,The Unemployment Club (2003),\N,2003,movie,0,98,nm0062463,"nm0062463,nm1401426","nm0387184,nm0858138,nm1121368,nm3345204,nm3962..."
44527,168566,"Taşkafa, Stories of the Street (2013)",\N,2013,movie,0,66,nm1601901,"nm0074237,nm1601901,nm1699957",nm1699957
44685,168908,Has the Film Already Started? (1951),\N,1951,movie,0,62,nm0501085,nm0501085,nm0501085


In [111]:
original_csv_ratings_data = config['original_csv_ratings_data']
ratings = pd.read_csv(file_depth + original_csv_ratings_data, encoding="UTF-8")
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [112]:
filtered_ratings_zero_genres = ratings[ratings['movieId'].isin(zero_values_genres['movieId'])]
filtered_ratings_zero_year = ratings[ratings['movieId'].isin(zero_values_year['movieId'])]
print(f"Number of ratings deleted because of null year information is {len(filtered_ratings_zero_year)} and because of null genre information {len(filtered_ratings_zero_genre)}.")

Number of ratings deleted because of null year information is 1 and because of null genre information 18.


We will delete records with null movieYear and genres information, since it is a small amount of records and number of ratings correstponding to these records is also small.

In [114]:
ratings = ratings[~ratings['movieId'].isin(filtered_ratings_zero_year['movieId'])]
ratings = ratings[~ratings['movieId'].isin(filtered_ratings_zero_genres['movieId'])]

In [115]:
movies_with_context = movies_with_context.drop(zero_values_year.index)

# Movie Checkmate is in both dataframes to delete, so we will remove it from the second dataframe. (index: 48645)
zero_values_genres.drop(48645, inplace=True)

movies_with_context = movies_with_context.drop(zero_values_genres.index)

C:\Users\adamb\AppData\Local\Temp\ipykernel_25500\2268483969.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zero_values_genres.drop(48645, inplace=True)


In [118]:
print("Number of records in movies_with_context after deletion: ", len(movies_with_context))
print("Number of records in ratings after deletion: ", len(ratings))

Number of records in movies_with_context after deletion:  62295
Number of records in ratings after deletion:  25000077


In [119]:
ratings.to_csv(file_depth + new_context_path + 'ratings.csv', index=False)

---
---
### Unite separation of genres

Unite separation of genres in genres column. We will replace the "," separator with "|" separator.

In [127]:
movies_with_context['genres'] = movies_with_context['genres'].str.replace(',', '|')

In [126]:
print('Number of null values in column genres:', movies_with_context['genres'].isna().sum())
print('Number of null values in column movieYear:', movies_with_context['movieYear'].isna().sum())
print('Number of null values in column titleType:', movies_with_context['titleType'].isna().sum())
print('Number of null values in column isAdult:', movies_with_context['isAdult'].isna().sum())
print('Number of null values in column runtimeMinutes:', movies_with_context['runtimeMinutes'].isna().sum())
print('Number of null values in column directors:', movies_with_context['directors'].isna().sum())
print('Number of null values in column writers:', movies_with_context['writers'].isna().sum())
print('Number of null values in column actor:', movies_with_context['actor'].isna().sum())

Number of null values in column genres: 0
Number of null values in column movieYear: 0
Number of null values in column titleType: 0
Number of null values in column isAdult: 0
Number of null values in column runtimeMinutes: 0
Number of null values in column directors: 0
Number of null values in column writers: 0
Number of null values in column actor: 6688


We are gonna replace all the NaN values in actor column with "no_actor"

In [132]:
movies_with_context['actor'].fillna("no_actor", inplace=True)
print('Number of null values in column actor:', movies_with_context['actor'].isna().sum())

Number of null values in column actor: 0


### Save the dataset

In [134]:
movies_with_context.to_csv(file_depth + new_context_path + 'movies_with_context.csv', index=False)